# File Input

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter
import os
import sys 
from sklearn.model_selection import ShuffleSplit, train_test_split, cross_val_score, StratifiedKFold

# py path只須執行一次
# sys.path.insert(0, r'C:\Users\oplab\Desktop\ML_final project')

from LauraProcess import *
import LauraProcess
import importlib
importlib.reload(LauraProcess)


from LauraProcess import LauraProcess

laura = LauraProcess()
# X_train, y_train, Test_IDs_df = laura.process()
# X_train, y_train, Test_IDs_df = laura.process_impute_knn()
X_train, y_train, Test_IDs_df = laura.process_impute_simple()

print('X_train=====================')
print(X_train)
print('y_train=====================')
print(y_train)
print('X_test======================')
print(Test_IDs_df)


train_raw_X = X_train.drop(['Customer ID'], axis=1)
train_raw_y = y_train
test_raw_X = Test_IDs_df.drop(['Customer ID'], axis=1)

X_train=====================
      Gender       Age  Under 30  Senior Citizen  Married  Dependents  \
0        0.0  0.295082       0.0             0.0      0.0         0.0   
1        0.0  0.573770       0.0             0.0      0.0         0.0   
2        0.0  0.081967       1.0             0.0      0.0         1.0   
3        0.0  0.065574       1.0             0.0      0.0         0.0   
4        1.0  0.360656       0.0             0.0      1.0         0.0   
...      ...       ...       ...             ...      ...         ...   
5629     0.0  0.131148       1.0             0.0      0.0         0.0   
5630     1.0  0.508197       0.0             0.0      0.0         0.0   
5631     0.0  0.672131       0.0             0.0      0.0         1.0   
5632     1.0  0.475410       0.0             0.0      0.0         1.0   
5633     0.0  0.852459       0.0             1.0      1.0         0.0   

      Number of Dependents  Population  Satisfaction Score  Referred a Friend  \
0            

In [2]:
# from sklearn.cluster import KMeans
# from sklearn import metrics
# import pandas as pd
# import numpy as np
# import matplotlib.pyplot as plt
# from collections import Counter


# files = ['location_v2.csv','satisfaction.csv','services_mean.csv','status_v2.csv']
# Train_IDs_df = pd.read_csv(r'C:\Users\oplab\Desktop\ML_final project\html2021final\Train_IDs.csv')
# Test_IDs_df = pd.read_csv(r'C:\Users\oplab\Desktop\ML_final project\html2021final\Test_IDs.csv')
# population_df = pd.read_csv(r'C:\Users\oplab\Desktop\ML_final project\html2021final\population.csv')

# df = pd.read_csv(r'C:\Users\oplab\Desktop\ML_final project\html2021final\demographics_full.csv')
# for file in files:
#     other = pd.read_csv(r'C:\Users\oplab\Desktop\ML_final project\html2021final\\'+file)
#     if 'Count' in other.columns:
#         other = other.drop(['Count'], axis=1)
#     col_file = file.replace('.csv','')
#     df = df.merge(other, how='outer', left_on='Customer ID', right_on='Customer ID',suffixes=(None, '_'+col_file))
#     if file =='location_v2.csv':
#         df = df.merge(population_df, how='left', left_on='Zip Code', right_on='Zip Code',suffixes=(None, '_'+col_file))
# # print(df)

# data_dum = pd.get_dummies(df.iloc[:,1:])
# data_dum = data_dum.drop(['Count','Zip Code','Latitude','Longitude'], axis=1)
# data_dum = data_dum.fillna(0)
# data_dum['Customer ID'] = df.iloc[:,0]
# print(data_dum)

# Process

In [4]:
# #train
# # Train_IDs_df = Train_IDs_df.merge(data_dum, how='left', left_on='Customer ID', right_on='Customer ID',suffixes=(None, '_'+col_file))
# # train_X = Train_IDs_df.drop(['Customer ID','Churn Category'], axis=1)
# train_df = Train_IDs_df.merge(data_dum, how='left', left_on='Customer ID', right_on='Customer ID',suffixes=(None, '_'+col_file))
# train_raw_X = train_df.drop(['Customer ID','Churn Category'], axis=1)
# train_raw_y = train_df['Churn Category']
# print(np.shape(train_raw_X))

# # test
# # Test_IDs_df = Test_IDs_df.merge(data_dum, how='left', left_on='Customer ID', right_on='Customer ID',suffixes=(None, '_'+col_file))
# # test_X = Test_IDs_df.drop(['Customer ID','Churn Category'], axis=1)
# test_df = Test_IDs_df.merge(data_dum, how='left', left_on='Customer ID', right_on='Customer ID',suffixes=(None, '_'+col_file))
# test_raw_X = test_df.drop(['Customer ID','Churn Category'], axis=1)
# print(np.shape(test_raw_X))
# # Train_IDs_df.head()
# # test_X.isna()

In [5]:
# X = train_X.drop(['Churn Category'], axis=1)
# y = train_y['Churn Category']

# Feature selection

In [5]:
from sklearn.feature_selection import VarianceThreshold,SelectKBest,chi2,SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.ensemble import ExtraTreesClassifier


def featureSelection(featureSelectionMethod):
    selected_train_X = train_raw_X
    if featureSelectionMethod == "VarianceThreshold":
        sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
        sel = sel.fit(train_raw_X)
        selected_train_X = sel.transform(train_raw_X)
        test_X = sel.transform(test_raw_X)
        selected_train_X.shape
        
    elif featureSelectionMethod == "SelectKBest":    
        m = SelectKBest(chi2, k=50).fit(train_raw_X, train_raw_y)
        selected_train_X = m.transform(train_raw_X)
        test_X = m.transform(test_raw_X)
        selected_train_X.shape
        
    elif featureSelectionMethod == "LinearSVC":         
        lsvc = LinearSVC(C=0.01, penalty="l2", dual=False).fit(train_raw_X, train_raw_y)
        model = SelectFromModel(lsvc, prefit=True)
        selected_train_X = model.transform(train_raw_X)
        test_X = model.transform(test_raw_X)
        selected_train_X.shape
        
    elif featureSelectionMethod == "ExtraTreesClassifier":
        clf = ExtraTreesClassifier(n_estimators=50)
        clf = clf.fit(train_raw_X, train_raw_y)
        clf.feature_importances_  
        model = SelectFromModel(clf, prefit=True)
        selected_train_X = model.transform(train_raw_X)
        test_X = model.transform(test_raw_X)
        selected_train_X.shape 
    
    return selected_train_X, test_X

=========== split training data to X_train and X_val ===========

# Imbalanced

In [1]:
# pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
# pip install -U imbalanced-learn

Note: you may need to restart the kernel to use updated packages.


In [6]:
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, SVMSMOTE 


def overSamplingMethod(selectedMethod, selected_train_X, train_raw_y):
    balanced_train_X, balanced_train_y = selected_train_X, train_raw_y
    if selectedMethod == "SMOTE":
        sm = SMOTE(random_state=42)
        balanced_train_X, balanced_train_y = sm.fit_resample(selected_train_X, train_raw_y)
        print('Resampled dataset shape %s' % Counter(balanced_train_y))
    
    elif selectedMethod == "ADASYN":    
        ada = ADASYN(random_state=42)
        balanced_train_X, balanced_train_y = ada.fit_resample(selected_train_X, train_raw_y)
        print('Resampled dataset shape %s' % Counter(balanced_train_y))
    
    elif selectedMethod == "BorderlineSMOTE":      
        sm = BorderlineSMOTE(random_state=42)
        balanced_train_X, balanced_train_y = sm.fit_resample(selected_train_X, train_raw_y)
        print('Resampled dataset shape %s' % Counter(balanced_train_y))
    
    elif selectedMethod == "SVMSMOTE":        
        sm = SVMSMOTE(random_state=42)
        balanced_train_X, balanced_train_y = sm.fit_resample(selected_train_X, train_raw_y)
        print('Resampled dataset shape %s' % Counter(balanced_train_y))
        
    return balanced_train_X, balanced_train_y

In [ ]:
# train_X = balanced_train_X
# train_y = balanced_train_y

# train_X = selected_train_X
# train_y = train_raw_y

# RandomForest

In [7]:
import numpy as np
import pandas as pd
from sklearn import model_selection, ensemble, preprocessing, metrics
from sklearn.pipeline import make_pipeline

def RF_Classifier(train_X, train_y, test_X, test_y=None, validation=True):
    train_X = train_X[train_y != -1]
    train_y = train_y[train_y != -1]
    
    if validation:
        test_X = test_X[test_y != -1]
        test_y = test_y[test_y != -1]
         
    forest = make_pipeline(ensemble.RandomForestClassifier(n_estimators = 100))
    # scores = model_selection.cross_val_score(forest, train_X, train_y, cv=5, scoring='f1_macro')
    # print(np.average(scores))

    forest = forest.fit(train_X, train_y)
    y_pred = forest.predict(test_X)
    
    if validation:
        f1 = metrics.f1_score(test_y, y_pred, average="macro")
        return f1
    else:
        RF_result = pd.DataFrame()
        RF_result['Customer ID'] = Test_IDs_df['Customer ID']
        RF_result['Churn Category'] = np.array(y_pred, dtype='int64')
        RF_result.to_csv("RF_result.csv",index=False)
        print("Finished output: RF_result.csv")

# 績效
# accuracy = metrics.accuracy_score(test_y, test_y_predicted)
# print(accuracy)


# XGBoost

In [ ]:
pip install xgboost

In [8]:
from xgboost.sklearn import XGBClassifier

def XGB_Classifier(train_X, train_y, test_X, test_y=None, validation=True):
    train_X = train_X[train_y != -1]
    train_y = train_y[train_y != -1]
    
    if validation:
        test_X = test_X[test_y != -1]
        test_y = test_y[test_y != -1]
        
    xgboostModel = make_pipeline(XGBClassifier(n_estimators=100, learning_rate= 0.3,use_label_encoder =False))
#     scores = model_selection.cross_val_score(xgboostModel, train_X, train_y, cv=5, scoring='f1_macro')
#     print(np.average(scores))

    xgboostModel = xgboostModel.fit(train_X, train_y)
    y_pred = xgboostModel.predict(test_X)
    
    if validation:
        f1 = metrics.f1_score(test_y, y_pred, average="macro")
        return f1
    else:
        XGB_result = pd.DataFrame()
        XGB_result['Customer ID'] = Test_IDs_df['Customer ID']
        XGB_result['Churn Category'] = y_pred
        XGB_result.to_csv("XGB_result.csv",index=False)
        print("Finished output: XGB_result.csv")

# Semi-supervised (XGB)

In [9]:
from sklearn.semi_supervised import SelfTrainingClassifier

def semi_XGB_Classifier(train_X, train_y, test_X, test_y=None, validation=True):
    if validation:
        test_X = test_X[test_y != -1]
        test_y = test_y[test_y != -1]
        
    xgboostModel = XGBClassifier(n_estimators=100, learning_rate= 0.3,use_label_encoder =False)
    self_training_model = make_pipeline(SelfTrainingClassifier(xgboostModel))
    self_training_model.fit(train_X, train_y)
    # y_pred = self_training_model.predict(train_X)
#     scores = model_selection.cross_val_score(self_training_model, train_X, train_y, cv=5, scoring='f1_macro')
#     print(np.average(scores))
    y_pred = self_training_model.predict(test_X)

    if validation:
        f1 = metrics.f1_score(test_y, y_pred, average="macro")
        return f1
    else:
        semi_XGB_result = pd.DataFrame()
        semi_XGB_result['Customer ID'] = Test_IDs_df['Customer ID']
        semi_XGB_result['Churn Category'] = y_pred
        semi_XGB_result.to_csv("semi_XGB_result.csv",index=False)
        print("Finished output: semi_XGB_result.csv")

In [14]:
if __name__ == '__main__':
    
    featureSelectionMethods = ["VarianceThreshold","SelectKBest","LinearSVC","ExtraTreesClassifier"]
    overSamplingMethods = ["SMOTE","ADASYN","BorderlineSMOTE","SVMSMOTE"]
    
    # gothrough all combination to choose the best
    score={}
    for SelectionMethod in featureSelectionMethods:
        selected_train_X, test_X = featureSelection(SelectionMethod)
        
        for overSampling in overSamplingMethods:
            # cross validation
            skf = StratifiedKFold(n_splits=5)
            RF_score, XBG_score, semi_score=0, 0, 0
            
            for train_index, test_index in skf.split(selected_train_X, train_raw_y):
                X_train, X_val = selected_train_X[train_index], selected_train_X[test_index]
                y_train, y_val = train_raw_y[train_index], train_raw_y[test_index]
                
                # overSampling
                balanced_train_X, balanced_train_y = overSamplingMethod(overSampling, X_train, y_train)
                
                # Classifier models
                RF_score += RF_Classifier(balanced_train_X, balanced_train_y, X_val, y_val, validation=True)
                XBG_score += XGB_Classifier(balanced_train_X, balanced_train_y, X_val, y_val, validation=True)
                semi_score += semi_XGB_Classifier(balanced_train_X, balanced_train_y, X_val, y_val, validation=True)
            
            print("SelectionMethod:",SelectionMethod,"overSampling:",overSampling)
            score[SelectionMethod+", "+overSampling+", RF"] = RF_score/5
            print("RF_score:",RF_score/5)
            score[SelectionMethod+", "+overSampling+", XGB"] = XBG_score/5
            print("XBG_score:",XBG_score/5)
            score[SelectionMethod+", "+overSampling+", semi"] = semi_score/5
            print("semi_score:",semi_score/5)
    

Resampled dataset shape Counter({1: 2494, 0: 2494, -1: 2494, 3: 2494, 4: 2494, 2: 2494, 5: 2494})
[02:18:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:19:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:19:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd lik

[02:20:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:20:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:20:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:21:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[02:22:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:22:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:22:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:22:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[02:24:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:24:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:24:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:24:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[02:26:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:26:29] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:26:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[02:26:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

...

[05:18:05] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:18:16] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Resampled dataset shape Counter({1: 2494, 0: 2494, -1: 2494, 2: 2494, 5: 2494, 4: 2494, 3: 2494})
[05:18:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd lik

Resampled dataset shape Counter({1: 2495, 0: 2495, -1: 2495, 2: 2495, 5: 2495, 4: 2495, 3: 2495})
[05:23:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:23:13] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:23:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd lik

[05:27:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:27:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:28:07] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:28:15] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[05:31:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:31:42] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:31:50] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:31:58] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

[05:35:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:35:25] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:35:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[05:35:40] WARNING: C:/Users/Administrator/workspace/xgboost-win64_

In [15]:
    score_dict = dict(sorted(score.items(), key=lambda item: item[1], reverse=True))
    dict_key = list(score_dict.keys())
    print("dict_key:",dict_key[0], "score:",score_dict[dict_key[0]])
    print("dict_key:",dict_key[1], "score:",score_dict[dict_key[1]])

dict_key: LinearSVC, BorderlineSMOTE, semi score: 0.3320977454035616
dict_key: LinearSVC, BorderlineSMOTE, XGB score: 0.3309801160318898


In [16]:
    #final output
    selected_train_X, test_X = featureSelection("LinearSVC")
    balanced_train_X, balanced_train_y = overSamplingMethod("BorderlineSMOTE", selected_train_X, train_raw_y)
#     RF_Classifier(balanced_train_X, balanced_train_y, test_X, validation=False)
#     XGB_Classifier(balanced_train_X, balanced_train_y, test_X, validation=False)
    semi_XGB_Classifier(balanced_train_X, balanced_train_y, test_X, validation=False)

C:\Users\oplab\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\oplab\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Resampled dataset shape Counter({1: 3118, 0: 3118, -1: 3118, 2: 3118, 5: 3118, 4: 3118, 3: 3118})
[12:26:17] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:26:51] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[12:27:31] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd lik

In [18]:
    #final output
    selected_train_X, test_X = featureSelection("LinearSVC")
    balanced_train_X, balanced_train_y = overSamplingMethod("BorderlineSMOTE", selected_train_X, train_raw_y)
#     RF_Classifier(balanced_train_X, balanced_train_y, test_X, validation=False)
    XGB_Classifier(balanced_train_X, balanced_train_y, test_X, validation=False)
#     semi_XGB_Classifier(balanced_train_X, balanced_train_y, test_X, validation=False)

C:\Users\oplab\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(
C:\Users\oplab\anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but SelectFromModel was fitted without feature names
  warnings.warn(


Resampled dataset shape Counter({1: 3118, 0: 3118, -1: 3118, 2: 3118, 5: 3118, 4: 3118, 3: 3118})
[12:34:21] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Finished output: XGB_result.csv


In [ ]:
dict_key: ExtraTreesClassifier, ADASYN, XGB score: 0.33856181687495907
dict_key: LinearSVC, SMOTE, semi score: 0.3328010734196246